In [ ]:
// Initialization / nearest expiry / getAtmStrike
const debug = false;
const axios = require('axios');
const fs = require('fs');
const unzipper = require('unzipper');
const { parse } = require('papaparse');
const moment = require('moment');
let { authparams } = require("./creds");
const { idxNameTokenMap, downloadCsv, filterAndMapDates, 
  identify_option_type, fetchSpotPrice, getStrike, calcVix } = require('./utils/customLibrary');

let globalBigInput = {
  filteredIndexCSV: undefined
}
let globalInput = {
  susertoken: '',
  secondSession: false,
  launchChildProcessApp: false,
  indexName: 'NIFTY',
  delayTime: 1000,
  ocGap: undefined,
  token: undefined,
  pickedExchange: undefined,
  inputOptTsym: undefined,
  WEEKLY_EXPIRY: undefined,
  MONTHLY_EXPIRY: undefined,
};
globalInput.token = idxNameTokenMap.get(globalInput.indexName);

let biasProcess = {
  optionChain: undefined,
  atmCallSymbol: undefined,
  callSubStr: undefined,
  atmPutSymbol: undefined,
  putSubStr: undefined,
  itmCallSymbol: undefined,
  itmCallStrikePrice: undefined,
  callSubStr: undefined,
  itmPutSymbol: undefined,
  itmPutStrikePrice: undefined,
  putSubStr: undefined,
  vix: undefined,
}
let biasOutput = { // N[46] 20155 (-20)
  tsym: '',
  bias: 0,
  deltaMove: 0,
  spotLP: 0
}

const delay = (ms) => new Promise(resolve => setTimeout(resolve, ms));

//websocket
let websocket;
let websocket_closed= false;
let intervalId;

let latestQuotes = {};
let latestOrders = {};

let positionProcess = {
  smallestTwoPositionsInView: undefined // [{tsym: 'NIFTY07DEC23P20850', lp: '1.55', netqty: '-800', s_prdt_ali: 'MIS'}, {...}]
}

async function findNearestExpiry() {
  let csvUrl, zipFilePath, csvFilePath;
  const exchangeType = globalInput.indexName.includes('EX') ? 'BFO' : 'NFO';
  csvUrl = `https://api.shoonya.com/${exchangeType}_symbols.txt.zip`;
  zipFilePath = `./${exchangeType}_symbols.zip`;
  csvFilePath = `./${exchangeType}_symbols.txt`;
  try {
    // Download and extract the CSV file
    await downloadCsv(csvUrl, zipFilePath);
    await fs.createReadStream(zipFilePath).pipe(unzipper.Extract({ path: '.' }));

    // Read CSV data into a JavaScript object
    const csvData = fs.readFileSync(csvFilePath, 'utf-8');
    const { data: symbolDf } = parse(csvData, { header: true });
    
    globalBigInput.filteredIndexCSV = filterAndMapDates(symbolDf.filter((row) => ['OPTIDX', 'FUTIDX'].includes(row.Instrument) && row.TradingSymbol.startsWith(globalInput.indexName)));
    // console.log(globalBigInput.filteredIndexCSV);
    // [
    //  {
    //   Exchange: 'NFO',
    //   Token: '72903',
    //   LotSize: '50',
    //   Symbol: 'NIFTY',
    //   TradingSymbol: 'NIFTY29FEB24P21750',
    //   Expiry: '2024-02-29',
    //   Instrument: 'OPTIDX',
    //   OptionType: 'PE',
    //   StrikePrice: '21750',
    //   TickSize: '0.05',
    //   '': ''
    // },
    //BFO,833613,15,BKXFUT,BANKEX24FEBFUT,26-FEB-2024,FUTIDX,XX,0,0.05,
    const expiryList = [...new Set(globalBigInput.filteredIndexCSV.filter((row) => row.Instrument === 'OPTIDX').map((row) => row.Expiry))];
    const expiryFutList = globalBigInput.filteredIndexCSV
      .filter((row) => row.Instrument === 'FUTIDX')
      .map((row) => ({ Exchange: row.Exchange, LotSize: row.LotSize, TradingSymbol: row.TradingSymbol, Expiry: row.Expiry }));
    expiryList.sort();
    expiryFutList.sort((a, b) => moment(a.Expiry).diff(moment(b.Expiry)));
    
    globalInput.inputOptTsym = [...new Set(globalBigInput.filteredIndexCSV.filter((row) => (row.Instrument === 'OPTIDX' && row.Expiry === expiryList[0])).map((row) => row.TradingSymbol))][0];
    globalInput.WEEKLY_EXPIRY = expiryList[0];
    globalInput.MONTHLY_EXPIRY = expiryFutList[0].Expiry;
    globalInput.ocGap = expiryFutList[0].LotSize;
    globalInput.pickedExchange = expiryFutList[0].Exchange;
  } catch (error) {
    console.error('Error:', error.message);
  } finally {
    // Clean up: Delete downloaded files
    fs.unlinkSync(zipFilePath);
    fs.unlinkSync(csvFilePath);
  }
};
// Execute the findNearestExpiry function
findNearestExpiry();

const Api = require("./lib/RestApi");
let api = new Api({});

const getAtmStrike = () => {
  const s = latestQuotes[`NSE|${globalInput.token}`];
  debug && console.log(s) //updateAtmStrike(s) --> 50, spot object -> s.lp = 20100
  return Math.round(s.lp / globalInput.ocGap) * globalInput.ocGap
}

In [ ]:
// login method
const { spawn } = require('child_process');
login = async (api) => {
    try {
        const res = await api.login(authparams);
        return true;
    } catch (err) {
        return false;
    }
};
executeLogin = async () => {
    const isLoggedIn = await login(api);
    if (globalInput.launchChildProcessApp){    
        const childProcess = spawn('node', ['app.js'], {
            stdio: ['pipe', 'ignore', 'ignore', 'ipc']  // 'ignore' for stdout and stderr
        });

        childProcess.on('message', (message) => {
            console.log(`Message from Child Process: ${message}`);
        });

        childProcess.on('close', (code) => {
            console.log(`Child process exited with code ${code}`);
        });

        // Send a message to the child process
        childProcess.send('Hello from the parent process!');
    }
    if (!isLoggedIn) {
        return;
    }
};


In [ ]:
// websocket
function receiveQuote(data) {
    // console.log("Quote ::", data);
    // Update the latest quote value for the corresponding instrument
    latestQuotes[data.e + '|' + data.tk] = data;
}

function receiveOrders(data) {
    // console.log("Order ::", data);
    // Update the latest order value for the corresponding instrument
    latestOrders[data.Instrument] = data;
}

function open(data) {
    // console.log(`NSE|${globalInput.token}`)
    const initialInstruments = [`NSE|${globalInput.token}`, 'NSE|26017']; 

    //vix:
    // {
    //     t: 'tf',
    //     e: 'NSE',
    //     tk: '26017',
    //     lp: '12.77',
    //     pc: '-7.06',
    //     ft: '1701931151'
    //   } latestQuotes['NSE|26017']
    subscribeToInstruments(initialInstruments);
    // console.log("Subscribing to :: ", initialInstruments);
}

function subscribeToInstruments(instruments) {
    instruments.forEach(instrument => {
        api.subscribe(instrument);
    });
}

function dynamicallyAddSubscription(newInstrument) {
    if (!latestQuotes[newInstrument]) {
        // console.log("Subscribing to :: ", newInstrument);
        api.subscribe(newInstrument);
    }
}

params = {
    'socket_open': open,
    'quote': receiveQuote,
    'order': receiveOrders
};
async function startWebsocket() {
    websocket = api.start_websocket(params);
    await delay(1000);
}

In [ ]:
// findITMfromOC
async function getOptionChain() {
    try {
        biasProcess.atmStrike = getAtmStrike();
        const optionChainResponse = await api.get_option_chain(globalInput.pickedExchange, globalInput.inputOptTsym, biasProcess.atmStrike, 2);
        // console.log(optionChainResponse, 'optionChainResponse')
        if (optionChainResponse.stat === 'Ok') {
            debug && console.log(optionChainResponse, 'optionChainResponse')
            return optionChainResponse;
        } else {
            console.error('Error getting option chain:', optionChainResponse);
            return null;
        }
    } catch (error) {
        console.error('Error:', error.message);
        return null;
    }
}

// // Function to find the ATM symbol from the option chain
// function findATMSymbol(optionType) {
//     // Filter options by type (CE for Call, PE for Put)
//     const filteredOptions = biasProcess.optionChain.values.filter(option => option.optt === optionType);

//     // Assume that the ATM option is the one with the nearest strike price to the spot price
//     const spotPrice = parseFloat(biasProcess.optionChain.values[0].strprc); // Use the first option's strike price as spotPrice
//     filteredOptions.sort((a, b) => Math.abs(parseFloat(a.strprc) - spotPrice) - Math.abs(parseFloat(b.strprc) - spotPrice));

//     // ATM symbol is the one with the minimum absolute difference
//     const atmSymbol = filteredOptions[0].tsym;
//     return atmSymbol;
// }

// async function findATMSymbolfromOC() {
//     await delay(1000)
//     // Get the Nifty option chain
//     biasProcess.optionChain = await getOptionChain();

//     if (biasProcess.optionChain) {
//         // Find the ATM symbol
//         biasProcess.atmCallSymbol = findATMSymbol('CE');
//         biasProcess.atmPutSymbol = findATMSymbol('PE');
        
//     }
// }

// Function to find the ATM symbol from the option chain
function findITMSymbol(optionType) {
    // Filter options by type (CE for Call, PE for Put)
    const filteredOptions = biasProcess.optionChain.values.filter(option => option.optt === optionType);

    // Assume that the ATM option is the one with the nearest strike price to the spot price
    const spotPrice = parseFloat(biasProcess.optionChain.values[0].strprc); // Use the first option's strike price as spotPrice
    // filteredOptions.sort((a, b) => Math.abs(parseFloat(a.strprc) - spotPrice) - Math.abs(parseFloat(b.strprc) - spotPrice));
    filteredOptions.sort((a, b) => a.tsym.localeCompare(b.tsym));

    // ATM symbol is the one with the minimum absolute difference
    debug && console.log(filteredOptions, 'filteredOptions sorted')
    
    const itmSymbol = filteredOptions[0].optt == 'PE' ? filteredOptions[3].tsym: filteredOptions[1].tsym
    
    return itmSymbol;
}

// Function to find the ATM symbol from the option chain
function findITMStrikePrice(optionType) {
    // Filter options by type (CE for Call, PE for Put)
    const filteredOptions = biasProcess.optionChain.values.filter(option => option.optt === optionType);

    // Assume that the ATM option is the one with the nearest strike price to the spot price
    const spotPrice = parseFloat(biasProcess.optionChain.values[0].strprc); // Use the first option's strike price as spotPrice
    // filteredOptions.sort((a, b) => Math.abs(parseFloat(a.strprc) - spotPrice) - Math.abs(parseFloat(b.strprc) - spotPrice));
    filteredOptions.sort((a, b) => a.tsym.localeCompare(b.tsym));

    // ATM symbol is the one with the minimum absolute difference
    debug && console.log(filteredOptions, 'filteredOptions sorted1')
    
    const itmStrikePrice = filteredOptions[0].optt == 'PE' ? filteredOptions[3].strprc: filteredOptions[1].strprc
    
    return itmStrikePrice;
}

async function updateITMSymbolfromOC() {
    await delay(1000)
    // Get the Nifty option chain
    biasProcess.optionChain = await getOptionChain();

    if (biasProcess.optionChain) {
        // Find the ITM symbol
        biasProcess.itmCallSymbol = findITMSymbol('CE');
        biasProcess.itmPutSymbol = findITMSymbol('PE');
        // Find the ITM strikePrice
        biasProcess.itmCallStrikePrice = findITMStrikePrice('CE');
        biasProcess.itmPutStrikePrice = findITMStrikePrice('PE');
        debug && console.log(biasProcess)
    }
}

In [ ]:
// Print the latest value of Bias every second
myRecurringFunction = async () => {

  getAtmStrike()!= biasProcess.atmStrike && await updateITMSymbolfromOC();
  biasProcess.vix = latestQuotes['NSE|26017']?.pc;
  // console.log(latestQuotes['NSE|26017'], "latestQuotes['NSE|26017']")
  debug && console.log(`${globalInput.indexName}:`, latestQuotes[`NSE|${globalInput.token}`] ? latestQuotes[`NSE|${globalInput.token}`].lp : "N/A", "Order:", latestOrders[`NSE|${globalInput.token}`]);
  debug && console.log(`${biasProcess.itmCallSymbol}:`, latestQuotes[biasProcess.callSubStr] ? latestQuotes[biasProcess.callSubStr].lp : "N/A", "Order:", latestOrders[biasProcess.callSubStr]);
  debug && console.log(`${biasProcess.itmPutSymbol}:`, latestQuotes[biasProcess.putSubStr] ? latestQuotes[biasProcess.putSubStr].lp : "N/A", "Order:", latestOrders[biasProcess.putSubStr]);

  ltpSuggestedPut = +biasProcess.itmPutStrikePrice - (+latestQuotes[biasProcess.putSubStr].lp);
  ltpSuggestedCall = (+latestQuotes[biasProcess.callSubStr].lp + +biasProcess.itmCallStrikePrice);

  biasOutput.bias = Math.round(((ltpSuggestedCall + ltpSuggestedPut) / 2) - latestQuotes[`NSE|${globalInput.token}`].lp);
  console.log(biasOutput.bias, ' :biasOutput.bias');   
  console.log(biasProcess.vix, ' :vix %');
  // Check a condition to determine whether to stop the recurring function
  if (websocket_closed) {
    clearInterval(intervalId);
    // console.log(latestOrders, 'latestOrders')
    console.log('Recurring function stopped.');
  }
}

function getTokenByTradingSymbol(tradingSymbol) {
  const option = biasProcess.optionChain.values.find(option => option.tsym === tradingSymbol);
  if (option) {
    return option.token;
  } else {
    return null; // TradingSymbol not found
  }
}

In [ ]:
// get bias run
getBias = async () => {
    try {
        await executeLogin();
        await startWebsocket();
        await updateITMSymbolfromOC();
        // Start the recurring function and store the interval identifier
        intervalId = setInterval(await myRecurringFunction, 1000);
        // Dynamically add a subscription after 10 seconds
        biasProcess.callSubStr = `NFO|${getTokenByTradingSymbol(biasProcess.itmCallSymbol)}`;
        biasProcess.putSubStr = `NFO|${getTokenByTradingSymbol(biasProcess.itmPutSymbol)}`;
        dynamicallyAddSubscription(biasProcess.callSubStr);
        dynamicallyAddSubscription(biasProcess.putSubStr);
        // setTimeout(() => {
        //     api.closeWebSocket();
        //     websocket_closed = true;
        // }, 10000);
    } catch (error) {
        console.error(error);
    }
};
// getBias();

In [ ]:
// getTwoPositionsSmallestLP
getTwoPositionsSmallestLP = async () => {
    await executeLogin();
    api.get_positions()
        .then((data) => { 
            console.log(data, 'data');
            const filteredData = data
                .filter(item => parseInt(item.netqty) < 0 && item.instname == 'OPTIDX')
                .map(({ tsym, lp, netqty, s_prdt_ali }) => ({ tsym, lp, netqty, s_prdt_ali }))
                .sort((a, b) => a.lp - b.lp) // Sort based on lp in ascending order
                .slice(0, 2); // Take the first two items with least lp

            positionProcess.smallestTwoPositionsInView = filteredData;
            // [
            //     {
            //       tsym: 'NIFTY07DEC23P20850',
            //       lp: '1.55',
            //       netqty: '-800',
            //       s_prdt_ali: 'MIS'
            //     },
            //     {
            //       tsym: 'NIFTY07DEC23C20950',
            //       lp: '2.60',
            //       netqty: '-800',
            //       s_prdt_ali: 'MIS'
            //     }
            //   ]
        });
}

getTwoPositionsSmallestLP();
